##### Set cwd

In [5]:
import os

print(os.getcwd())
os.chdir(r'D:\Coding\bodies-at-rest\PressurePose')
print(os.getcwd())

import sys
print(sys.executable)

D:\Coding\bodies-at-rest\PressurePose
D:\Coding\bodies-at-rest\PressurePose
d:\Coding\bodies-at-rest\env\python.exe


##### Variables

In [6]:
red_verts = False
seg_limbs = False

##### Constants

In [7]:
MAT_WIDTH = 0.762  # metres
MAT_HEIGHT = 1.854  # metres
MAT_HALF_WIDTH = MAT_WIDTH / 2
NUMOFTAXELS_X = 64  # 73 #taxels
NUMOFTAXELS_Y = 27  # 30
NUMOFOUTPUTDIMS = 3
NUMOFOUTPUTNODES_TRAIN = 24
NUMOFOUTPUTNODES_TEST = 10
INTER_SENSOR_DISTANCE = 0.0286  # metres
LOW_TAXEL_THRESH_X = 0
LOW_TAXEL_THRESH_Y = 0
HIGH_TAXEL_THRESH_X = (NUMOFTAXELS_X - 1)
HIGH_TAXEL_THRESH_Y = (NUMOFTAXELS_Y - 1)
TEST_SUBJECT = 9
CAM_BED_DIST = 1.66

##### Libraries

In [8]:
import pickle as pickle
import numpy as np
from scipy import ndimage
import torch
import time
import lib_pyrender_basic as libPyRender
from chumpy.ch import MatVecMult
import chumpy
import chumpy as ch

In [9]:
os.chdir('../')
print(os.getcwd())
from smpl.smpl_webuser.posemapper import posemap
from smpl.smpl_webuser.verts import verts_core
from smpl.smpl_webuser import lbs
os.chdir('PressurePose')
print(os.getcwd())

D:\Coding\bodies-at-rest
D:\Coding\bodies-at-rest\PressurePose


##### File paths

In [10]:
filepath_prefix = '../data_BR/synth/'
GENDER = "f"

#Replace this with some subset of data of your choice
TESTING_FILENAME = 'quick_test/test_rollpi_'+GENDER+'_lay_set23to24_3000_qt_convnet_1_anglesDC_184000ct_128b_x1pm_tnh_clns10p_100e_'+str(0.00002)+'lr'

database_file_f = []
database_file_m = []

if GENDER == "f":
    database_file_f.append(filepath_prefix+TESTING_FILENAME+'.p')
    model_pkl_file_path = '../smpl/models/basicModel_f_lbs_10_207_0_v1.0.0.pkl'
else:
    database_file_m.append(filepath_prefix+TESTING_FILENAME+'.p')
    model_pkl_file_path = '../smpl/models/basicModel_m_lbs_10_207_0_v1.0.0.pkl'


print(f'database_file_f: {database_file_f}')
print(f'database_file_m: {database_file_m}')

database_file_f: ['../data_BR/synth/quick_test/test_rollpi_f_lay_set23to24_3000_qt_convnet_1_anglesDC_184000ct_128b_x1pm_tnh_clns10p_100e_2e-05lr.p']
database_file_m: []


# 1. class __PysicalTrainer__()  

## 1.1. def __init__()

### 1.1.1. CTRL_PNL Variables

In [11]:
CTRL_PNL = {}
CTRL_PNL['loss_vector_type'] = 'anglesDC'
CTRL_PNL['verbose'] = False
CTRL_PNL['batch_size'] = 1
CTRL_PNL['num_epochs'] = 100
CTRL_PNL['incl_inter'] = True
CTRL_PNL['shuffle'] = True
CTRL_PNL['incl_ht_wt_channels'] = False
CTRL_PNL['incl_pmat_cntct_input'] = True
CTRL_PNL['dropout'] = False
CTRL_PNL['lock_root'] = False
CTRL_PNL['num_input_channels'] = 2
CTRL_PNL['GPU'] = False
CTRL_PNL['regr_angles'] = False
CTRL_PNL['aws'] = False
CTRL_PNL['depth_map_labels'] = True #can only be true if we have 100% synthetic data for training
CTRL_PNL['depth_map_labels_test'] = True #can only be true is we have 100% synth for testing
CTRL_PNL['depth_map_output'] = CTRL_PNL['depth_map_labels']
CTRL_PNL['depth_map_input_est'] = False  #do this if we're working in a two-part regression
CTRL_PNL['adjust_ang_from_est'] = CTRL_PNL['depth_map_input_est'] #holds betas and root same as prior estimate
CTRL_PNL['clip_sobel'] = True
CTRL_PNL['clip_betas'] = True
CTRL_PNL['mesh_bottom_dist'] = True
CTRL_PNL['full_body_rot'] = True
CTRL_PNL['normalize_input'] = False
CTRL_PNL['normalize_per_image'] = False
CTRL_PNL['all_tanh_activ'] = True
CTRL_PNL['L2_contact'] = True
CTRL_PNL['pmat_mult'] = int(1)
CTRL_PNL['cal_noise'] = False
CTRL_PNL['double_network_size'] = False
CTRL_PNL['first_pass'] = True


weight_joints = 1.0#opt.j_d_ratio*2
weight_depth_planes = (1-0.5)#*2

if CTRL_PNL['cal_noise'] == True:
    CTRL_PNL['incl_pmat_cntct_input'] = False #if there's calibration noise we need to recompute this every batch
    CTRL_PNL['clip_sobel'] = False

if CTRL_PNL['incl_pmat_cntct_input'] == True:
    CTRL_PNL['num_input_channels'] += 1
if CTRL_PNL['depth_map_input_est'] == True: #for a two part regression
    CTRL_PNL['num_input_channels'] += 3
CTRL_PNL['num_input_channels_batch0'] = np.copy(CTRL_PNL['num_input_channels'])
if CTRL_PNL['incl_ht_wt_channels'] == True:
    CTRL_PNL['num_input_channels'] += 2
if CTRL_PNL['cal_noise'] == True:
    CTRL_PNL['num_input_channels'] += 1

pmat_std_from_mult = ['N/A', 11.70153502792190, 19.90905848383454, 23.07018866032369, 0.0, 25.50538629767412]
if CTRL_PNL['cal_noise'] == False:
    sobel_std_from_mult = ['N/A', 29.80360490415032, 33.33532963163579, 34.14427844692501, 0.0, 34.86393494050921]
else:
    sobel_std_from_mult = ['N/A', 45.61635847182483, 77.74920396659292, 88.89398421073700, 0.0, 97.90075708182506]

CTRL_PNL['norm_std_coeffs'] =  [1./41.80684362163343,  #contact
                                        1./16.69545796387731,  #pos est depth
                                        1./45.08513083167194,  #neg est depth
                                        1./43.55800622930469,  #cm est
                                        1./pmat_std_from_mult[int(CTRL_PNL['pmat_mult'])], #pmat
                                        1./sobel_std_from_mult[int(CTRL_PNL['pmat_mult'])], #pmat sobel
                                        1./1.0,                #bed height mat
                                        1./1.0,                #OUTPUT DO NOTHING
                                        1./1.0,                #OUTPUT DO NOTHING
                                        1. / 30.216647403350,  #weight
                                        1. / 14.629298141231]  #height


CTRL_PNL['filepath_prefix'] = '/home/henry/'
    #CTRL_PNL['filepath_prefix'] = '/media/henry/multimodal_data_2/'

mat_size = (NUMOFTAXELS_X, NUMOFTAXELS_Y)

### 1.1.2. Prepare Training Data

##### Functions()

In [12]:
def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f, encoding='latin1')

#### 1. TensorPrepLib().load_files_to_database()

In [13]:
def load_files_to_database(database_file, creation_type, verbose = False, reduce_data = False, test = False):
    # load in the training or testing files.  This may take a while.
    dat = None
    for some_subject in database_file:
        #print creation_type, some_subject, 'some subject'
        if creation_type in some_subject:
            dat_curr = load_pickle(some_subject)
            print(some_subject, dat_curr['bed_angle_deg'][0])
            for key in dat_curr:
                if np.array(dat_curr[key]).shape[0] != 0:
                    for inputgoalset in np.arange(len(dat_curr['images'])):
                        datcurr_to_append = dat_curr[key][inputgoalset]
                        if key == 'images' and np.shape(datcurr_to_append)[0] == 3948:
                            datcurr_to_append = list(
                                np.array(datcurr_to_append).reshape(84, 47)[10:74, 10:37].reshape(1728))
                        try:
                            if test == False:
                                if reduce_data == True:
                                    if inputgoalset < len(dat_curr['images'])/4:
                                        dat[key].append(datcurr_to_append)
                                else:
                                    dat[key].append(datcurr_to_append)
                            else:
                                if len(dat_curr['images']) == 3000:
                                    if inputgoalset < len(dat_curr['images'])/2:
                                        dat[key].append(datcurr_to_append)
                                elif len(dat_curr['images']) == 1500:
                                    if inputgoalset < len(dat_curr['images'])/3:
                                        dat[key].append(datcurr_to_append)
                                else:
                                    dat[key].append(datcurr_to_append)

                        except:
                            try:
                                dat[key] = []
                                dat[key].append(datcurr_to_append)
                            except:
                                dat = {}
                                dat[key] = []
                                dat[key].append(datcurr_to_append)

    if dat is not None and verbose == True:
        for key in dat:
            print('all data keys and shape', key, np.array(dat[key]).shape)
    return dat

In [14]:
dat_f = load_files_to_database(database_file = database_file_f, creation_type = 'synth', reduce_data = False)
dat_m = load_files_to_database(database_file = database_file_m, creation_type = 'synth', reduce_data = False)

print(f'dat_f.keys(): {dat_f.keys()}')
print(f'dat_m: {dat_m}')

KeyboardInterrupt: 

#### 2. TensorPrepLib().prep_images()

In [ ]:
def prep_images(train_x_flat, dat_f, dat_m, num_repeats):
    for dat in [dat_f, dat_m]:
        if dat is not None:
            for entry in range(len(dat['images'])):
                for i in range(num_repeats):
                    train_x_flat.append(dat['images'][entry])
    return train_x_flat

In [ ]:
train_x_flat = []
train_x_flat = prep_images(train_x_flat, dat_f, dat_m, num_repeats = 1)
train_x_flat = list(np.clip(np.array(train_x_flat) * float(CTRL_PNL['pmat_mult']), a_min=0, a_max=100))

print(f'np.array(train_x_flat).shape: {np.array(train_x_flat).shape}')

#### 3. TensorPrepLib().prep_depth_contact()

In [ ]:
def prep_depth_contact(depth_contact_maps, dat_f, dat_m, num_repeats):
    for dat in [dat_f, dat_m]:
        if dat is not None:
            for entry in range(len(dat['images'])):
                for i in range(num_repeats):
                    depth_contact_maps.append([dat['mesh_depth'][entry], dat['mesh_contact'][entry]])
    return depth_contact_maps

In [ ]:
if CTRL_PNL['depth_map_labels'] == True:
    depth_contact_maps = [] # Initialize the precomputed depth and contact maps. only synth has this label.
    depth_contact_maps = prep_depth_contact(depth_contact_maps, dat_f, dat_m, num_repeats = 1)
else:
    depth_contact_maps = None

print(f'np.array(depth_contact_maps).shape: {np.array(depth_contact_maps).shape}')

#### 4. TensorPrepLib().prep_depth_contact_input_est()

In [ ]:
def prep_depth_contact_input_est(depth_contact_maps_input_est, dat_f, dat_m, num_repeats):
    for dat in [dat_f, dat_m]:
        if dat is not None:
            for entry in range(len(dat['images'])):
                for i in range(num_repeats):
                    mdm_est_pos = np.copy(dat['mdm_est'][entry])
                    mdm_est_neg = np.copy(dat['mdm_est'][entry])
                    mdm_est_pos[mdm_est_pos < 0] = 0
                    mdm_est_neg[mdm_est_neg > 0] = 0
                    mdm_est_neg *= -1
                    # depth_contact_input_est_list.append([dat['mdm_est'][entry], dat['cm_est'][entry]])
                    depth_contact_maps_input_est.append([mdm_est_pos, mdm_est_neg, dat['cm_est'][entry]*100])
    return depth_contact_maps_input_est

In [ ]:
if CTRL_PNL['depth_map_input_est'] == True:
    depth_contact_maps_input_est = [] # Initialize the precomputed depth and contact map input estimates
    depth_contact_maps_input_est = prep_depth_contact_input_est(depth_contact_maps_input_est, dat_f, dat_m, num_repeats = 1)
else:
    depth_contact_maps_input_est = None

print(f'CTRL_PNL[depth_map_input_est]: {CTRL_PNL["depth_map_input_est"]}')
print(f'np.array(depth_contact_maps_input_est).shape: {np.array(depth_contact_maps_input_est).shape}')

#### 5. PreprocessingLib().preprocessing_create_pressure_angle_stack()

In [ ]:
def preprocessing_create_pressure_angle_stack(train_x_flat, mat_size, CTRL_PNL):
    '''This is for creating a 2-channel input using the height of the bed. '''

    if CTRL_PNL['verbose']: print(np.max(train_x_flat))
    train_x_flat = np.clip(train_x_flat, 0, 100)

    print("normalizing per image", CTRL_PNL['normalize_per_image'])

    train_xa = []
    for map_index in range(len(train_x_flat)):
        # print map_index, mat_size, 'mapidx'
        # Resize mat to make into a matrix

        p_map = np.reshape(train_x_flat[map_index], mat_size)

        if CTRL_PNL['normalize_per_image'] == True:
            p_map = p_map * (20000./np.sum(p_map))

        if mat_size == (84, 47):
            p_map = p_map[10:74, 10:37]

        # this makes a sobel edge on the image
        sx = ndimage.sobel(p_map, axis=0, mode='constant')
        sy = ndimage.sobel(p_map, axis=1, mode='constant')
        p_map_inter = np.hypot(sx, sy)
        if CTRL_PNL['clip_sobel'] == True:
            p_map_inter = np.clip(p_map_inter, a_min=0, a_max = 100)

        if CTRL_PNL['normalize_per_image'] == True:
            p_map_inter = p_map_inter * (20000. / np.sum(p_map_inter))

        #print np.sum(p_map), 'sum after norm'
        train_xa.append([p_map, p_map_inter])

    return train_xa

In [ ]:
# stack the bed height array on the pressure image as well as a sobel filtered image
train_xa = preprocessing_create_pressure_angle_stack(train_x_flat, mat_size, CTRL_PNL)

print(f'np.array(train_xa).shape: {np.array(train_xa).shape}')

#### 6. TensorPrepLib().append_input_depth_contact()

In [ ]:
def append_input_depth_contact(train_xa, CTRL_PNL, depth_contact_maps_input_est = None, depth_contact_maps = None):
    if CTRL_PNL['incl_pmat_cntct_input'] == True:
        train_contact = np.copy(train_xa[:, 0:1, :, :]) #get the pmat contact map
        train_contact[train_contact > 0] = 100.

    if CTRL_PNL['depth_map_input_est'] == True:
        depth_contact_maps_input_est = np.array(depth_contact_maps_input_est)
        train_xa = np.concatenate((depth_contact_maps_input_est, train_xa), axis = 1)

    print(np.shape(train_xa), CTRL_PNL['incl_pmat_cntct_input'])
    if CTRL_PNL['incl_pmat_cntct_input'] == True:
        train_xa = np.concatenate((train_contact, train_xa), axis=1)

    print(np.shape(train_xa))
    if CTRL_PNL['depth_map_labels'] == True:
        depth_contact_maps = np.array(depth_contact_maps) #GROUND TRUTH
        train_xa = np.concatenate((train_xa, depth_contact_maps), axis=1)

    return train_xa

In [ ]:
# stack the depth and contact mesh images (and possibly a pmat contact image) together
train_xa_ = append_input_depth_contact(np.array(train_xa), CTRL_PNL, depth_contact_maps_input_est, depth_contact_maps)

print(f'train_xa_.shape: {train_xa_.shape}')

#### 7. TensorPrepLib().prep_labels()

In [ ]:
def prep_labels(y_flat, dat, num_repeats, z_adj, gender, is_synth, loss_vector_type, initial_angle_est, full_body_rot = False):
    # printing
    print(f'y_flat_arr.shape = {np.array(y_flat).shape}')
    print(f'dat_array.shape = {np.array(dat).shape}')
    print(f'z_adj = {z_adj}')
    print(f'gender = {gender}')
    print(f'is_synth = {is_synth}')
    print(f'loss_vector_type = {loss_vector_type}')
    print(f'initial_angle_est = {initial_angle_est}')
    print(f'full_body_rot = {full_body_rot}')


    if gender == "f":
        g1 = 1
        g2 = 0
    elif gender == "m":
        g1 = 0
        g2 = 1
    if is_synth == True:
        s1 = 1
    else:
        s1 = 0
    z_adj_all = np.array(24 * [0.0, 0.0, z_adj*1000])
    z_adj_one = np.array(1 * [0.0, 0.0, z_adj*1000])

    if is_synth == True and loss_vector_type != 'direct':
        if dat is not None:
            for entry in range(len(dat['markers_xyz_m'])):
                c = np.concatenate((dat['markers_xyz_m'][entry][0:72] * 1000 + z_adj_all,
                                    dat['body_shape'][entry][0:10],
                                    dat['joint_angles'][entry][0:72],
                                    dat['root_xyz_shift'][entry][0:3] + np.array([0.0, 0.0, z_adj]),
                                    [g1], [g2], [s1],
                                    [dat['body_mass'][entry]],
                                    [(dat['body_height'][entry]-1.)*100],), axis=0)  # [x1], [x2], [x3]: female synth: 1, 0, 1.
                if initial_angle_est == True:
                    c = np.concatenate((c,
                                        dat['betas_est'][entry][0:10],
                                        dat['angles_est'][entry][0:72],
                                        dat['root_xyz_est'][entry][0:3]), axis = 0)
                    if full_body_rot == True:
                        c = np.concatenate((c, dat['root_atan2_est'][entry][0:6]), axis = 0)
                for i in range(num_repeats):
                    y_flat.append(c)

    elif is_synth == True and loss_vector_type == 'direct':
        if dat is not None:
            for entry in range(len(dat['markers_xyz_m_offset'])):
                c = np.concatenate((np.array(9 * [0]),
                                    dat['markers_xyz_m_offset'][entry][3:6] * 1000 + z_adj_one,  # TORSO
                                    # fixed_torso_markers,  # TORSO
                                    dat['markers_xyz_m_offset'][entry][21:24] * 1000 + z_adj_one,  # L KNEE
                                    dat['markers_xyz_m_offset'][entry][18:21] * 1000 + z_adj_one,  # R KNEE
                                    np.array(3 * [0]),
                                    dat['markers_xyz_m_offset'][entry][27:30] * 1000 + z_adj_one,  # L ANKLE
                                    dat['markers_xyz_m_offset'][entry][24:27] * 1000 + z_adj_one,  # R ANKLE
                                    np.array(18 * [0]),
                                    dat['markers_xyz_m_offset'][entry][0:3] * 1000 + z_adj_one,  # HEAD
                                    # fixed_head_markers,
                                    np.array(6 * [0]),
                                    dat['markers_xyz_m_offset'][entry][9:12] * 1000 + z_adj_one,  # L ELBOW
                                    dat['markers_xyz_m_offset'][entry][6:9] * 1000 + z_adj_one,  # R ELBOW
                                    dat['markers_xyz_m_offset'][entry][15:18] * 1000 + z_adj_one,  # L WRIST
                                    dat['markers_xyz_m_offset'][entry][12:15] * 1000 + z_adj_one,  # R WRIST
                                    np.array(6 * [0]),
                                    np.array(85 * [0]),
                                    [g1], [g2], [s1],
                                    [dat['body_mass'][entry]],
                                    [(dat['body_height'][entry]-1.)*100],), axis=0)  # [x1], [x2], [x3]: female synth: 1, 0, 1.
                if initial_angle_est == True:
                    c = np.concatenate((c,
                                        dat['betas_est'][entry][0:10],
                                        dat['angles_est'][entry][0:72],
                                        dat['root_xyz_est'][entry][0:3]), axis = 0)
                for i in range(num_repeats):
                    y_flat.append(c)


    elif is_synth == False:
        if dat is not None:
            for entry in range(len(dat['markers_xyz_m'])):
                c = np.concatenate((np.array(9 * [0]),
                                    dat['markers_xyz_m'][entry][3:6] * 1000,  # TORSO
                                    # fixed_torso_markers,  # TORSO
                                    dat['markers_xyz_m'][entry][21:24] * 1000,  # L KNEE
                                    dat['markers_xyz_m'][entry][18:21] * 1000,  # R KNEE
                                    np.array(3 * [0]),
                                    dat['markers_xyz_m'][entry][27:30] * 1000,  # L ANKLE
                                    dat['markers_xyz_m'][entry][24:27] * 1000,  # R ANKLE
                                    np.array(18 * [0]),
                                    dat['markers_xyz_m'][entry][0:3] * 1000,  # HEAD
                                    # fixed_head_markers,
                                    np.array(6 * [0]),
                                    dat['markers_xyz_m'][entry][9:12] * 1000,  # L ELBOW
                                    dat['markers_xyz_m'][entry][6:9] * 1000,  # R ELBOW
                                    dat['markers_xyz_m'][entry][15:18] * 1000,  # L WRIST
                                    dat['markers_xyz_m'][entry][12:15] * 1000,  # R WRIST
                                    np.array(6 * [0]),
                                    np.array(85 * [0]),
                                    [g1], [g2], [s1],
                                    [dat['body_mass'][entry]],
                                    [(dat['body_height'][entry]-1.)*100],), axis=0)  # [x1], [x2], [x3]: female real: 1, 0, 0.
                if initial_angle_est == True:
                    c = np.concatenate((c,
                                        dat['betas_est'][entry][0:10],
                                        dat['angles_est'][entry][0:72],
                                        dat['root_xyz_est'][entry][0:3]), axis = 0)
                for i in range(num_repeats):
                    y_flat.append(c)

    elif is_synth == 'real_nolabels':
        s1 = 1
        WEIGHT_LBS = 190.
        HEIGHT_IN = 73.
        weight_input = WEIGHT_LBS/2.20462
        height_input = (HEIGHT_IN*0.0254 - 1)*100

        if dat is not None:
            for entry in range(len(dat['images'])):
                c = np.concatenate((np.array(157 * [0]),
                                    [g1], [g2], [s1],
                                    [weight_input],
                                    [height_input],), axis=0)  # [x1], [x2], [x3]: female synth: 1, 0, 1.
                if initial_angle_est == True:
                    c = np.concatenate((c,
                                        dat['betas_est'][entry][0:10],
                                        dat['angles_est'][entry][0:72],
                                        dat['root_xyz_est'][entry][0:3]), axis = 0)
                    if full_body_rot == True:
                        c = np.concatenate((c, dat['root_atan2_est'][entry][0:6]), axis = 0)
                for i in range(num_repeats):
                    y_flat.append(c)


    return y_flat

In [ ]:
train_y_flat = []

train_y_flat = prep_labels(y_flat = train_y_flat, dat = dat_f, num_repeats = 1,
                           z_adj = -0.075, gender = "f", is_synth = True,
                           loss_vector_type = CTRL_PNL['loss_vector_type'],
                           initial_angle_est = CTRL_PNL['adjust_ang_from_est'],
                           full_body_rot = CTRL_PNL['full_body_rot'])

print()
print(f'np.array(train_y_flat).shape: {np.array(train_y_flat).shape}')
print()

train_y_flat = prep_labels(y_flat = train_y_flat, dat = dat_m, num_repeats = 1,
                           z_adj = -0.075, gender = "m", is_synth = True,
                            loss_vector_type = CTRL_PNL['loss_vector_type'],
                            initial_angle_est = CTRL_PNL['adjust_ang_from_est'],
                            full_body_rot = CTRL_PNL['full_body_rot'])

print()
print(f'np.array(train_y_flat).shape: {np.array(train_y_flat).shape}')

#### 8. Normalize the input and labels, if __CTRL_PNL['normalize_input'] == True__

In [ ]:
def normalize_network_input(train_xa, CTRL_PNL):

    if CTRL_PNL['depth_map_input_est'] == True:
        normalizing_std_constants = CTRL_PNL['norm_std_coeffs']

        if CTRL_PNL['cal_noise'] == True:
            normalizing_std_constants = normalizing_std_constants[1:] # here we don't precompute the contact

        for i in range(train_xa.shape[1]):
            train_xa[:, i, :, :] *= normalizing_std_constants[i]

    else:
        normalizing_std_constants = []
        normalizing_std_constants.append(CTRL_PNL['norm_std_coeffs'][0])
        normalizing_std_constants.append(CTRL_PNL['norm_std_coeffs'][4])
        normalizing_std_constants.append(CTRL_PNL['norm_std_coeffs'][5])
        normalizing_std_constants.append(CTRL_PNL['norm_std_coeffs'][6])
        normalizing_std_constants.append(CTRL_PNL['norm_std_coeffs'][7])

        if CTRL_PNL['cal_noise'] == True:
            normalizing_std_constants = normalizing_std_constants[1:] # here we don't precompute the contact

        for i in range(train_xa.shape[1]):
            print("normalizing idx", i)
            train_xa[:, i, :, :] *= normalizing_std_constants[i]

        # for i in range(x.shape[0]):
        #    print torch.m

    return train_xa

In [ ]:
def normalize_wt_ht(train_y_flat, CTRL_PNL):
    # normalizing_std_constants = [1./30.216647403349857, 1./14.629298141231091]
    train_y_flat = np.array(train_y_flat)
    #y[:, 160] *= normalizing_std_constants[0]
    #y[:, 161] *= normalizing_std_constants[1]
    train_y_flat[:, 160] *= CTRL_PNL['norm_std_coeffs'][8]
    train_y_flat[:, 161] *= CTRL_PNL['norm_std_coeffs'][9]

    return train_y_flat

In [ ]:
if CTRL_PNL['normalize_input'] == True:
    train_xa = normalize_network_input(train_xa, CTRL_PNL)

In [ ]:
if CTRL_PNL['normalize_input'] == True:
    train_y_flat = normalize_wt_ht(train_y_flat, CTRL_PNL)


#### 9. Convert the input (train_xa) & labels (train_y_flat) to Tensor

In [ ]:
train_x_tensor = torch.Tensor(train_xa_)

print(f'train_xa_.shape: {train_xa_.shape}')
print(f'train_x_tensor.shape: {train_x_tensor.shape}')

In [ ]:
train_y_tensor = torch.Tensor(train_y_flat)

print(f'np.array(train_y_flat).shape: {np.array(train_y_flat).shape}')
print(f'train_y_tensor.shape: {train_y_tensor.shape}')

#### 10. Prepare Dataset & DataLoader

In [ ]:
train_dataset = torch.utils.data.TensorDataset(train_x_tensor, train_y_tensor)
train_loader = torch.utils.data.DataLoader(train_dataset, CTRL_PNL['batch_size'], shuffle=CTRL_PNL['shuffle'])

#### 11. Print;  
- input(x) & output(y) shapes  
- Dataset & DataLoader lengths  
- Model path

In [ ]:
print(f'train_x_tensor.shape: {train_x_tensor.shape}')
print(f'train_y_tensor.shape: {train_y_tensor.shape}')
print(f': {model_pkl_file_path}')

print(f'train_dataset len: {len(train_dataset)}')
print(f'train_loader len: {len(train_loader)}')

## 1.2. def __val_general__():

### __model = load_model()__

#### __dd = ready_arguments()__

In [ ]:
def ready_arguments(model_pkl_file_path):
    if not isinstance(model_pkl_file_path, dict):
        with open(model_pkl_file_path, 'rb') as f:
            dd = pickle.load(f, encoding='latin1')
    else:
        dd = model_pkl_file_path

#     backwards_compatibility_replacements(dd)
# def backwards_compatibility_replacements(dd):
    # replacements
    if 'default_v' in dd:
        dd['v_template'] = dd['default_v']
        del dd['default_v']
    if 'template_v' in dd:
        dd['v_template'] = dd['template_v']
        del dd['template_v']
    if 'joint_regressor' in dd:
        dd['J_regressor'] = dd['joint_regressor']
        del dd['joint_regressor']
    if 'blendshapes' in dd:
        dd['posedirs'] = dd['blendshapes']
        del dd['blendshapes']
    if 'J' not in dd:
        dd['J'] = dd['joints']
        del dd['joints']
#     backwards_compatibility_replacements(dd)


    # defaults
    if 'bs_style' not in dd:
        dd['bs_style'] = 'lbs'    
        
    want_shapemodel = 'shapedirs' in dd
    nposeparms = dd['kintree_table'].shape[1]*3

    if 'trans' not in dd:
        dd['trans'] = np.zeros(3)
    if 'pose' not in dd:
        dd['pose'] = np.zeros(nposeparms)
    if 'shapedirs' in dd and 'betas' not in dd:
        dd['betas'] = np.zeros(dd['shapedirs'].shape[-1])

    for s in ['v_template', 'weights', 'posedirs', 'pose', 'trans', 'shapedirs', 'betas', 'J']:
        if (s in dd) and not hasattr(dd[s], 'dterms'):
            dd[s] = ch.array(dd[s])

    if want_shapemodel:
        dd['v_shaped'] = dd['shapedirs'].dot(dd['betas'])+dd['v_template']
        v_shaped = dd['v_shaped']
        J_tmpx = MatVecMult(dd['J_regressor'], v_shaped[:,0])        
        J_tmpy = MatVecMult(dd['J_regressor'], v_shaped[:,1])        
        J_tmpz = MatVecMult(dd['J_regressor'], v_shaped[:,2])        
        dd['J'] = ch.vstack((J_tmpx, J_tmpy, J_tmpz)).T    
        dd['v_posed'] = v_shaped + dd['posedirs'].dot(posemap(dd['bs_type'])(dd['pose']))
    else:    
        dd['v_posed'] = dd['v_template'] + dd['posedirs'].dot(posemap(dd['bs_type'])(dd['pose']))
            
    return dd

In [ ]:
dd = ready_arguments(model_pkl_file_path)

In [ ]:
# print output of the loaded pickle file

print(f'os.getcwd(): {os.getcwd()}')
os.chdir('../')
from z_utils import print_dict
os.chdir('PressurePose')
print(f'os.getcwd(): {os.getcwd()}')

print_dict(dd)

#### remaining code from __load_model()__

In [ ]:
args = {
    'pose': dd['pose'],
    'v': dd['v_posed'],
    'J': dd['J'],
    'weights': dd['weights'],
    'kintree_table': dd['kintree_table'],
    'xp': ch,
    'want_Jtr': True,
    'bs_style': dd['bs_style']
}

In [ ]:
model, Jtr = verts_core(**args)

In [ ]:
print(f'model.shape: {model.shape}')
print(f'Jtr.shape: {Jtr.shape}')

In [ ]:
model = model + dd['trans'].reshape((1,3))
model.J_transformed = Jtr + dd['trans'].reshape((1,3))

for k, v in list(dd.items()):
    setattr(model, k, v)

### End __load_model()__

In [ ]:
pyRender = libPyRender.pyRenderMesh(render = True)

In [ ]:
# This will loop a total = training_images/batch_size times
for batch_idx, batch in enumerate(train_loader):

    if batch_idx in [0, 1]: continue

    betas_gt = torch.mean(batch[1][:, 72:82], dim = 0).numpy()
    angles_gt = torch.mean(batch[1][:, 82:154], dim = 0).numpy()
    root_shift_est_gt = torch.mean(batch[1][:, 154:157], dim = 0).numpy()

    pmat = batch[0][0, 1, :, :].clone().numpy() # * 11.70153502792190

    for beta in range(betas_gt.shape[0]):
        model.betas[beta] = betas_gt[beta]

    for angle in range(angles_gt.shape[0]):
        model.pose[angle] = angles_gt[angle]


    # ground truth human mesh vertices
    smpl_verts_gt = np.array(model.r)

    for s in range(root_shift_est_gt.shape[0]):
        smpl_verts_gt[:, s] += (root_shift_est_gt[s] - float(model.J_transformed[0, s]))

    smpl_verts_gt = np.concatenate(
        (smpl_verts_gt[:, 1:2] - 0.286 + 0.0143,
            smpl_verts_gt[:, 0:1] - 0.286 + 0.0143,
            0.0 - smpl_verts_gt[:, 2:3]), axis=1)

    # mesh faces
    smpl_faces = np.array(model.f)

    # this gets the joint cartesian positions. no code here to visualize.
    joint_cart_gt = np.array(model.J_transformed).reshape(24, 3)
    for s in range(root_shift_est_gt.shape[0]):
        joint_cart_gt[:, s] += (root_shift_est_gt[s] - float(model.J_transformed[0, s]))



    if red_verts == False:
    # this code renders the whole ground truth mesh.
        pyRender.render_3D_data(camera_point = None, pmat = pmat, smpl_verts_gt = smpl_verts_gt,
                                        smpl_faces = smpl_faces, segment_limbs = seg_limbs)
    else:
        # this code renders only camera-facing vertices in the mesh
        camera_point = [1.09898028, 0.46441343, -CAM_BED_DIST]
        pyRender.render_3D_data(camera_point=camera_point, pmat=pmat, smpl_verts_gt=smpl_verts_gt,
                                        smpl_faces=smpl_faces, segment_limbs = seg_limbs)

    # time.sleep(3)